In [1]:
import json
from pathlib import Path

# import dotenv
# dotenv.load_dotenv("/home/oscar/nlitp/eva_proof_server/.env")

import sentence_transformers  # dependency for sentence_feature_extractor.py
# check that we can use sentence_transformers
print(sentence_transformers.SentenceTransformer('all-MiniLM-L6-v2').encode(['hello world']).shape)
# import openai  # dependency for chatgpt.py
import evadb
cursor = evadb.connect(evadb_dir=Path("./eva_db")).cursor()

/home/oscar/.cache/pypoetry/virtualenvs/steam-games-mWAKNGq7-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/oscar/.cache/pypoetry/virtualenvs/steam-games-mWAKNGq7-py3.10/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


(1, 384)


In [3]:
# Register the function as embed text
cursor.query("""
DROP FUNCTION IF EXISTS EmbedText
""").df()
cursor.query("""
CREATE FUNCTION EmbedText IMPL './sentence_feature_extractor.py'
""").df()
cursor.query("""
SELECT EmbedText('hello world')
""").df()

,features
0,"[[-0.03447732, 0.031023178, 0.0067349477, 0.02..."


In [49]:
# add the data to evadb from games.db
# we will just copy the data from games.db to evadb
# create the games table
cursor.query("""
DROP TABLE IF EXISTS games
""").df()
cursor.query("""
CREATE TABLE games (
            name TEXT(0),
            img_url TEXT(0),
            date TEXT(0),
            developer TEXT(0),
            full_desc TEXT(0),
            price TEXT(0),
            url_info TEXT(0)
            )"""
).df()
cursor.query("""
SELECT * FROM games
""").df()


""


In [50]:

# copy the data from games.db to evadb
# connect to games.db with sqlite3
import sqlite3
conn = sqlite3.connect("games.db")
c = conn.cursor()
# get the data from games.db
c.execute("SELECT * FROM games")
games = c.fetchall()
# insert the data into evadb
from tqdm import tqdm   
for i, game in tqdm(enumerate(games)):
    if i > 1000:
        break
    game = list(game)
    for i in range(len(game)):
        game[i] = game[i].replace("'", "").replace('"', '').replace(";", "").replace(":", "")#.replace("(", "").replace(")", "")
    cursor.query(f"""
    INSERT INTO games (name, img_url, date, developer, full_desc, price, url_info)
    VALUES ('{game[0]}', '{game[1]}', '{game[2]}', '{game[3]}', '{game[4]}', '{game[5]}', '{game[6]}')
    """).df()
# check that the data is there
out = cursor.query("""
SELECT * FROM games
""").df()

1001it [00:29, 33.89it/s]


In [2]:
cursor.query("""
SELECT * FROM games
""").df().iloc[100].to_dict()

{'_row_id': 101,
 'name': 'Among Us - Mini Crewmate Bundle',
 'img_url': 'https//steamcdn-a.akamaihd.net/steam/apps/971996/header.jpg?t=1559936187',
 'date': 'Jun 7, 2019',
 'developer': 'Innersloth',
 'full_desc': 'About This Content  A mini you! A robotic friend? Includes the Mini Crewmate and Robot! (Only one pet can be equipped at a time) Support Among Us servers and Innersloth!',
 'price': '299',
 'url_info': 'https//store.steampowered.com/app/971996/Among_Us__Mini_Crewmate_Bundle/?snr=1_7_7_230_150_1'}

In [55]:
# create a vector index using faiss
cursor.query("""
DROP INDEX IF EXISTS game_vector_idx
""").df()
# CREATE INDEX [index_name]
# ON [table_name] ([function_name]([column_name]))
# USING [index_method]
cursor.query("""
CREATE INDEX game_vector_idx
ON games (EmbedText(games.full_desc))
USING FAISS
""").df()

11-18-2023 16:44:18 WARNING[drop_object_executor:drop_object_executor.py:_handle_drop_index:0113] Index game_vector_idx does not exist, therefore cannot be dropped.
/home/oscar/.cache/pypoetry/virtualenvs/steam-games-mWAKNGq7-py3.10/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py:157: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentences_sorted = [sentences[idx] for idx in length_sorted_idx]


,0
0,Index game_vector_idx successfully added to th...


In [ ]:
# manually 

In [2]:
test_query = "fast-paced action game"
cursor.query(f"""
SELECT * FROM games ORDER BY Similarity(EmbedText('{test_query}'), EmbedText(full_desc)) LIMIT 05;
""").df()

,_row_id,name,img_url,date,developer,full_desc,price,url_info
0,538,Pistol Whip,https//steamcdn-a.akamaihd.net/steam/apps/1079...,"Nov 7, 2019",Cloudhead Games ltd.,About This Game Inspired by God-mode action mo...,249,https//store.steampowered.com/app/1079800/Pist...
1,464,Combat Mission Shock Force 2,https//steamcdn-a.akamaihd.net/steam/apps/1369...,"Aug 31, 2020",Battlefront,About This Game The latest title in the famous...,599,https//store.steampowered.com/app/1369370/Comb...
2,388,Zero Caliber VR,https//steamcdn-a.akamaihd.net/steam/apps/8772...,"Nov 9, 2018",XREAL Games,About This Game ABOUT THIS GAMEZero Caliber ...,249,https//store.steampowered.com/app/877200/Zero_...
3,951,Need for Speed™ Most Wanted,https//steamcdn-a.akamaihd.net/steam/apps/1262...,"Oct 30, 2012",Criterion Games,About This Game The open-world action in Need ...,499,https//store.steampowered.com/app/1262560/Need...
4,857,Need for Speed™ Payback,https//steamcdn-a.akamaihd.net/steam/apps/1262...,"Nov 6, 2017",Ghost Games,About This Game Set in the underworld of Fortu...,499,https//store.steampowered.com/app/1262580/Need...


In [2]:
# create a simple flask endpoint to serve the five most similar games
from flask import Flask, request, jsonify
app = Flask(__name__)

# allow cross origin requests from any domain
from flask_cors import CORS
CORS(app)

@app.route('/get_similar_games', methods=['GET'])
def get_similar_games():
    query = request.args.get('query')
    out = cursor.query(f"""
    SELECT * FROM games ORDER BY Similarity(EmbedText('{query}'), EmbedText(full_desc)) LIMIT 15;
    """).df()
    out_recs = out.to_dict(orient="records")
    # replace every url with https with https: (BC EVADB) doesn't let me insert w/ ?
    for rec in out_recs:
        rec["img_url"] = rec["img_url"].replace("https", "https:")
        rec["url_info"] = rec["url_info"].replace("https", "https:")
    return jsonify(out_recs)

app.run(host="localhost", port=5000)

# this can be accessed as follows: http://localhost:5000/get_similar_games?query=fast-paced%20action%20game

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [30/Nov/2023 16:19:07] "GET /get_similar_games?query=vision%20and%20robotics HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2023 16:19:17] "GET /get_similar_games?query=robotics HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2023 16:25:27] "GET /get_similar_games?query=big%20data%20analysis HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2023 16:25:32] "GET /get_similar_games?query=robots HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2023 16:25:40] "GET /get_similar_games?query=About%20This%20Game%20War%20is%20raging,%20pilot!%20Are%20you%20ready%20for%20surprise%20attacks,%20intricate%20tactical%20maneuvers%20and%20the%20many%20sneaky%20tricks%20your%20rivals%20have%20in%20store%20for%20you?%20Destroy%20enemy%20robots,%20capture%20all%20the%20beacons,%20and%20upgrade%20your%20weapons%20to%20increase%20your%20battle%20robots%20combat%20strength,%20speed%20and%20durability.%20Prove%20yourself%20in%20each%20map%20and%20use%20different%20strategies%20and